# Obtén datos de la API de Spotify

![imagen](img/spotify.png)

Te tendrás que [registrar](https://developer.spotify.com/) y consultar la [documentación](https://developer.spotify.com/documentation/web-api)

Queremos que consultes a la api para que te devuelva todas las canciones de todos los discos de tu grupo o cantante favorito.

Pasos:

- Autentícate en la API de Spotify usando tus credenciales (client ID, client secret y redirect URI). **hecho**
- Busca el artista por nombre usando el endpoint /v1/search para obtener su artist_id. **hecho**
- Usa el artist_id para consultar todos sus álbumes con el endpoint /v1/artists/{id}/albums.
- Recorre todas las páginas de resultados (porque la API devuelve máximo 50 álbumes por petición).
- Para cada álbum, llama al endpoint /v1/albums/{id}/tracks para obtener las canciones.
- Guarda o muestra los resultados, combinando el nombre del álbum y el de cada canción.

meter esto en un script


Aquí te dejamos el código en python para poder empezar a hacer las llamadas, rellenando algunas variables como tus keys.

In [12]:
#pip install spotipy

# SCRIPT FINAL

In [ ]:
import os
import spotipy 
import pandas as pd

from spotipy.oauth2 import SpotifyOAuth

# Se borra el token previamente almacenado en cache para evitar problemas
if os.path.exists(".cache"):
    os.remove(".cache")

# Autenticación con clave publica privada
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id="meter aqui client_id",
    client_secret="meter aqui client_secret",
    redirect_uri="meter aqui uri"
))

# Petición a la API para conseguir el ID del artista
resultado_artista = sp.search(q='artist:Bob Marley', type='artist', limit=1)
dict_artista = resultado_artista["artists"]["items"][0]
print("Nombre:", dict_artista["name"])
print("ID:", dict_artista["id"])

id_artista = dict_artista["id"]

# Inicialización de la paginación (50 entradas max)
pag_actual_albumes = sp.artist_albums(id_artista, limit=50)

# Lista vacía donde guardan los diccionarios sacados del JSON para "items"
lista_dict_albumes = []

# Se guarda la primera página de 50 entradas antes del while
lista_dict_albumes.extend(pag_actual_albumes["items"])

# Bucle de paginación, mientras exista etiqueta next con url para la siguiente página de 50 albumes
# se pasa a la siguiente página con la función next de spotipy
while pag_actual_albumes["next"]:
    pag_actual_albumes = sp.next(pag_actual_albumes)
    lista_dict_albumes.extend(pag_actual_albumes["items"])

# Lista vacía donde guardan los diccionarios sacados del JSON para "items"
lista_tracks = []

for album in lista_dict_albumes:
    album_id = album["id"] 
    album_name = album["name"]

    # Inicialización de la paginación (50 entradas max)
    all_tracks = sp.album_tracks(album_id, limit = 50)

    while all_tracks["next"]:
        for track in all_tracks["items"]:
            lista_tracks.append({
                "canción": track["name"],
                "álbum": album_name
            })
        all_tracks = sp.next(all_tracks)

    # El while no entra en la última página porque next = None
    for track in all_tracks["items"]:
        lista_tracks.append({
            "canción": track["name"],
            "álbum": album_name
        })

# Conversion de la lista de diccionarios a dataframe, es un objeto sencillo de pasar a dt
df_tracks_marley = pd.DataFrame(lista_tracks)
print(df_tracks_marley)

# Conversion del dt a archivo csv y Excel
df_tracks_marley.to_csv("canciones_bob_marley.csv", sep = ';', index = True)
df_tracks_marley.to_excel('canciones_bob_marley.xlsx')

Nombre: Bob Marley & The Wailers
ID: 2QsynagSdAqZj3U9HgDzjD
                                                canción  \
0                                      Get Up, Stand Up   
1                                   Roots, Rock, Reggae   
2                                    I Shot The Sheriff   
3                                       No More Trouble   
4     War/No More Trouble - From "Bob Marley: One Lo...   
...                                                 ...   
5381                                       Play It Cool   
5382                                         Zion Gates   
5383                                  Get in the Groove   
5384                                        Tribulation   
5385                                          Want More   

                                              álbum  
0     One Love (Original Motion Picture Soundtrack)  
1     One Love (Original Motion Picture Soundtrack)  
2     One Love (Original Motion Picture Soundtrack)  
3     One Love 

# PRUEBAS

## Importaciones necesarias.

In [47]:
import os
import spotipy
import json
import pandas as pd

from spotipy.oauth2 import SpotifyOAuth

In [33]:
import os
if os.path.exists(".cache"):
    os.remove(".cache")

In [34]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id="c5944d02f10f40b0b0086753f2558b42",
    client_secret="9039c08fe50641e5a083bb41bce0775a",
    redirect_uri="http://127.0.0.1:8000"
))

In [35]:
resultado_artista = sp.search(q='artist:Bob Marley', type='artist', limit=1)
resultado_artista

{'artists': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=1&query=artist%3ABob%20Marley&type=artist',
  'limit': 1,
  'next': 'https://api.spotify.com/v1/search?offset=1&limit=1&query=artist%3ABob%20Marley&type=artist',
  'offset': 0,
  'previous': None,
  'total': 19,
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2QsynagSdAqZj3U9HgDzjD'},
    'followers': {'href': None, 'total': 13375171},
    'genres': ['reggae', 'roots reggae'],
    'href': 'https://api.spotify.com/v1/artists/2QsynagSdAqZj3U9HgDzjD',
    'id': '2QsynagSdAqZj3U9HgDzjD',
    'images': [{'url': 'https://i.scdn.co/image/b5aae2067db80f694a980e596e7f49618c1206c9',
      'height': 858,
      'width': 1000},
     {'url': 'https://i.scdn.co/image/4cd57e5e12ea2c24644c40886d65a9b22eca9802',
      'height': 549,
      'width': 640},
     {'url': 'https://i.scdn.co/image/02fd758d9805ef44d1caafc35ff17a47f9dff098',
      'height': 172,
      'width': 200},
     {'url': 'https://i.scdn.co/i

In [36]:
"""
Spotify te devuelve un diccionario (JSON) con esta estructura general:
{
  "artists": {
    "items": [
      {
        "id": "2QsynagSdAqZj3U9HgDzjD",
        "name": "Bob Marley & The Wailers",
        "genres": ["reggae"],
        "followers": {"total": 14593902},
        "popularity": 85
      }
    ],
    "limit": 1,
    "offset": 0,
    "total": 1
  }
}

"""
dict_artista = resultado_artista["artists"]["items"][0]
print("Nombre:", dict_artista["name"])
print("ID:", dict_artista["id"])

id_artista = dict_artista["id"]
print(id_artista)

Nombre: Bob Marley & The Wailers
ID: 2QsynagSdAqZj3U9HgDzjD
2QsynagSdAqZj3U9HgDzjD


In [37]:
artista_albums = sp.artist_albums(id_artista, limit=50)
artista_albums["items"]

[{'album_type': 'album',
  'total_tracks': 17,
  'available_markets': ['AR',
   'AU',
   'AT',
   'BE',
   'BO',
   'BR',
   'BG',
   'CA',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DK',
   'DE',
   'EC',
   'EE',
   'SV',
   'FI',
   'FR',
   'GR',
   'GT',
   'HN',
   'HK',
   'HU',
   'IS',
   'IE',
   'IT',
   'LV',
   'LT',
   'LU',
   'MY',
   'MT',
   'MX',
   'NL',
   'NZ',
   'NI',
   'NO',
   'PA',
   'PY',
   'PE',
   'PH',
   'PL',
   'PT',
   'SG',
   'SK',
   'ES',
   'SE',
   'CH',
   'TW',
   'TR',
   'UY',
   'US',
   'GB',
   'AD',
   'LI',
   'MC',
   'ID',
   'JP',
   'TH',
   'VN',
   'RO',
   'IL',
   'ZA',
   'SA',
   'AE',
   'BH',
   'QA',
   'OM',
   'KW',
   'EG',
   'MA',
   'DZ',
   'TN',
   'LB',
   'JO',
   'PS',
   'IN',
   'KZ',
   'MD',
   'UA',
   'AL',
   'BA',
   'HR',
   'ME',
   'MK',
   'RS',
   'SI',
   'KR',
   'BD',
   'PK',
   'LK',
   'GH',
   'KE',
   'NG',
   'TZ',
   'UG',
   'AM',
   'BZ',
   'BT',
   'BW',
   'BF',
   'CV',
   'C

In [ ]:
pag_actual_albumes = sp.artist_albums(id_artista, limit=50)
lista_dict_albumes = []
lista_dict_albumes.extend(pag_actual_albumes["items"])
while pag_actual_albumes["next"] != None:
    pag_actual_albumes = sp.next(pag_actual_albumes)
    lista_dict_albumes.extend(pag_actual_albumes["items"])

print(lista_dict_albumes)

"""
dict_albumes = {}

for album in lista_dict_albumes:
    dict_albumes[album["id"]] = album["name"]
print(dict_albumes)
serie_albumes = pd.Series(dict_albumes)
print(serie_albumes)
df_albumes = serie_albumes.reset_index()
df_albumes.columns = ["album_id", "album_name"]
print(df_albumes)
"""



KeyboardInterrupt: 

In [67]:
pag_actual_albumes = sp.artist_albums(id_artista, limit=50)
lista_dict_albumes = []
lista_dict_albumes.extend(pag_actual_albumes["items"])

while pag_actual_albumes["next"] != None:
    pag_actual_albumes = sp.next(pag_actual_albumes)
    lista_dict_albumes.extend(pag_actual_albumes["items"])

lista_tracks = []

for album in lista_dict_albumes:
    album_id = album["id"] 
    album_name = album["name"]
    all_tracks = sp.album_tracks(album_id)
    # print(album_id, ":", album_name)
    #print(tracks["items"])

    for track in all_tracks["items"]:
        lista_tracks.append({
            "canción": track["name"],
            "álbum": album_name
        })

df_tracks_marley = pd.DataFrame(lista_tracks)
df_tracks_marley

,canción,álbum
0,"Get Up, Stand Up",One Love (Original Motion Picture Soundtrack)
1,"Roots, Rock, Reggae",One Love (Original Motion Picture Soundtrack)
2,I Shot The Sheriff,One Love (Original Motion Picture Soundtrack)
3,No More Trouble,One Love (Original Motion Picture Soundtrack)
4,"War/No More Trouble - From ""Bob Marley: One Lo...",One Love (Original Motion Picture Soundtrack)
...,...,...
5149,Play It Cool,Legends & Anthems Essentials
5150,Zion Gates,Legends & Anthems Essentials
5151,Get in the Groove,Legends & Anthems Essentials
5152,Tribulation,Legends & Anthems Essentials


In [70]:
df_tracks_marley.to_csv("canciones_bob_marley.csv", sep = ';', index = True)
df_tracks_marley.to_excel('canciones_bob_marley.xlsx')